<div>
<img src='https://upload.wikimedia.org/wikipedia/commons/6/64/Sharif_University_Logo.jpg' alt="SUT logo" width=220 height=220 align=left class="saturate">

<br>
<font face="Times New Roman">
<div dir=ltr align=center>
<!-- <font color=0F5298 size=7> -->
<font color=0F5298 size=6>
    Introduction to Machine Learning <br> <br>
<!-- <font color=2565AE size=5> -->
<font size=5>
    Computer Engineering Department <br>
    Spring 2023 <br> <br>
<font color=606060 size=5>
    Homeworks 8 & 9 (Combined): Practical - Faces <br> <br>
<font color=686880 size=4>
    TAs: Arman Malekzadeh - Amirhossein Ramazani Bonab - Yalda Shabanzadeh
    
____

### Full Name : Mohammadreza Ahmadi Teshnizi
### Student Number : 98170646
### Colab Link: https://colab.research.google.com/drive/1dXaHIa3q2BVYPB0ayHe9mFru5yrDM9bu?usp=sharing
___

In this assignment, you'll be dealing with a dataset consisting of 400 faces, belonging to 40 people. Your main task is to train models capable of recognizing those faces.
The faces are `jpg` images packed together as a zip file called `images.zip`, and the `meta.csv` determines from which person each image was taken.

In [187]:
!wget -O images.zip https://www.dropbox.com/s/wrias0bjyte3rr2/images.zip?dl=1
!wget -O meta.csv https://www.dropbox.com/s/vqt1v8sfmz18rcf/faces-metadata.csv?dl=1

--2023-06-27 19:55:20--  https://www.dropbox.com/s/wrias0bjyte3rr2/images.zip?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.6.18, 2620:100:6019:18::a27d:412
Connecting to www.dropbox.com (www.dropbox.com)|162.125.6.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/dl/wrias0bjyte3rr2/images.zip [following]
--2023-06-27 19:55:20--  https://www.dropbox.com/s/dl/wrias0bjyte3rr2/images.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc251f45a8576a6ddeba53b5cfe7.dl.dropboxusercontent.com/cd/0/get/B-zmJDg-9CGyssn96iZd4Hb3ysX0hB6o_9OVKTHG17WyhoNmy2WviECkQhY7H2YFA0_A-PYNxsXxXsAs6X1c-P-URPBq8OlcB20q5UzWLnp-NU8ZptvMrSBqQEa4w49d31dPt8bPkB0GP80afgx-aCSQMhIK5FNCD-156REsW8KHISaF7VuaoCNh_98UxZJQpKE/file?dl=1# [following]
--2023-06-27 19:55:20--  https://uc251f45a8576a6ddeba53b5cfe7.dl.dropboxusercontent.com/cd/0/get/B-zmJDg-9CGyssn96iZd4Hb3ysX0hB6o_9OVKTHG17WyhoNmy2WviECkQhY7H2Y

In [188]:
# you can use the following packages
from PIL import Image
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.cluster import KMeans
from sklearn.decomposition import TruncatedSVD
import plotly.express as px
import zipfile
import os

# Sec 1: Data Preparation

Load all images as a numpy array (called `x`) and the metadata as a dataframe **(P1-1: 25 points)**

In [189]:
with zipfile.ZipFile('images.zip', 'r') as zip_ref:
    zip_ref.extractall('images')

metadata = pd.read_csv('meta.csv')

image_files = os.listdir('images/content/img-data')

x = []
for file in metadata["path"]:
    image_path = f"images/content/img-data/{file}"
    image = Image.open(image_path)
    image = np.array(image)
    x.append(image)

x = np.array(x)

In [190]:
x

array([[[ 82,  96, 108, ...,  97,  87,  81],
        [ 86, 100, 112, ...,  96,  84,  76],
        [ 88, 101, 116, ...,  97,  81,  74],
        ...,
        [ 51,  49,  54, ...,  42,  41,  42],
        [ 52,  50,  55, ...,  40,  39,  40],
        [ 53,  51,  54, ...,  38,  37,  37]],

       [[113, 121, 133, ...,  49,  52,  44],
        [119, 123, 132, ...,  51,  55,  49],
        [127, 128, 135, ...,  50,  54,  52],
        ...,
        [197, 198, 199, ...,  37,  37,  37],
        [196, 198, 200, ...,  33,  33,  33],
        [197, 199, 202, ...,  38,  38,  38]],

       [[ 80, 105, 125, ..., 108,  92,  79],
        [ 75,  98, 123, ..., 107,  91,  79],
        [ 66,  88, 121, ..., 105,  90,  79],
        ...,
        [ 34,  18,  26, ...,  35,  35,  38],
        [ 32,  17,  25, ...,  35,  36,  38],
        [ 32,  17,  24, ...,  36,  36,  39]],

       ...,

       [[124, 138, 154, ...,  80,  57,  55],
        [122, 137, 156, ...,  76,  52,  48],
        [121, 137, 156, ...,  71,  50,  44

In [191]:
metadata

,path,label
0,0.jpg,jones
1,1.jpg,jones
2,2.jpg,jones
3,3.jpg,jones
4,4.jpg,jones
...,...,...
395,395.jpg,vaux
396,396.jpg,vaux
397,397.jpg,vaux
398,398.jpg,vaux


Flatten and normalize the images **(P1-2: 10 points)**

Note: For this part, normalization only means dividing by 255.

In [192]:
x_flat = x.reshape(x.shape[0], -1)
x_normalized = x_flat / 255.0
x = x_normalized

In [193]:
x_normalized

array([[0.32156863, 0.37647059, 0.42352941, ..., 0.14901961, 0.14509804,
        0.14509804],
       [0.44313725, 0.4745098 , 0.52156863, ..., 0.14901961, 0.14901961,
        0.14901961],
       [0.31372549, 0.41176471, 0.49019608, ..., 0.14117647, 0.14117647,
        0.15294118],
       ...,
       [0.48627451, 0.54117647, 0.60392157, ..., 0.16862745, 0.17254902,
        0.17254902],
       [0.20392157, 0.24313725, 0.20784314, ..., 0.56862745, 0.58039216,
        0.58823529],
       [0.49411765, 0.48627451, 0.26666667, ..., 0.3372549 , 0.35686275,
        0.38823529]])

Encode the labels using `LabelEncoder` and save the result as a numpy array called `y` **(P1-3: 5 points)**

In [194]:
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(metadata['label'])
y = np.array(y)

In [195]:
y

array([17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 33, 33, 33, 33, 33, 33, 33,
       33, 33, 33,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2, 37, 37, 37, 37,
       37, 37, 37, 37, 37, 37, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 11,
       11, 11, 11, 11, 11, 11, 11, 11, 11,  5,  5,  5,  5,  5,  5,  5,  5,
        5,  5, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 18, 18, 18, 18, 18,
       18, 18, 18, 18, 18,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1, 19, 19,
       19, 19, 19, 19, 19, 19, 19, 19,  9,  9,  9,  9,  9,  9,  9,  9,  9,
        9, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 10, 10, 10, 10, 10, 10,
       10, 10, 10, 10, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29,  6,  6,  6,
        6,  6,  6,  6,  6,  6,  6, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30,
        3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  4,  4,  4,  4,  4,  4,  4,
        4,  4,  4, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 28, 28, 28, 28,
       28, 28, 28, 28, 28, 28, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 24,
       24, 24, 24, 24, 24

Split the data into training (80%), validation (10%), and testing (10%) parts. **(P1-4: 10 points)**

Note: The split procedure should maintain the class balance. This is sometimes called "[stratified sampling](https://en.wikipedia.org/wiki/Stratified_sampling)".

In [196]:
from sklearn.model_selection import train_test_split

X_train_val, X_test, y_train_val, y_test = train_test_split(x, y, test_size=0.1, stratify=y, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.111, stratify=y_train_val, random_state=42)

In [197]:
import numpy as np

print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)

print("X_val shape:", X_val.shape)
print("y_val shape:", y_val.shape)

print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)


unique_labels, label_counts = np.unique(y_train, return_counts=True)
print("Label distribution in y_train:", dict(zip(unique_labels, label_counts)))

unique_labels, label_counts = np.unique(y_val, return_counts=True)
print("Label distribution in y_val:", dict(zip(unique_labels, label_counts)))

unique_labels, label_counts = np.unique(y_test, return_counts=True)
print("Label distribution in y_test:", dict(zip(unique_labels, label_counts)))


print("Minimum value in X_train:", np.min(X_train))
print("Maximum value in X_train:", np.max(X_train))


X_train shape: (320, 4096)
y_train shape: (320,)
X_val shape: (40, 4096)
y_val shape: (40,)
X_test shape: (40, 4096)
y_test shape: (40,)
Label distribution in y_train: {0: 8, 1: 8, 2: 8, 3: 8, 4: 8, 5: 8, 6: 8, 7: 8, 8: 8, 9: 8, 10: 8, 11: 8, 12: 8, 13: 8, 14: 8, 15: 8, 16: 8, 17: 8, 18: 8, 19: 8, 20: 8, 21: 8, 22: 8, 23: 8, 24: 8, 25: 8, 26: 8, 27: 8, 28: 8, 29: 8, 30: 16, 31: 8, 32: 8, 33: 8, 34: 8, 35: 8, 36: 8, 37: 8, 38: 8}
Label distribution in y_val: {0: 1, 1: 1, 2: 1, 3: 1, 4: 1, 5: 1, 6: 1, 7: 1, 8: 1, 9: 1, 10: 1, 11: 1, 12: 1, 13: 1, 14: 1, 15: 1, 16: 1, 17: 1, 18: 1, 19: 1, 20: 1, 21: 1, 22: 1, 23: 1, 24: 1, 25: 1, 26: 1, 27: 1, 28: 1, 29: 1, 30: 2, 31: 1, 32: 1, 33: 1, 34: 1, 35: 1, 36: 1, 37: 1, 38: 1}
Label distribution in y_test: {0: 1, 1: 1, 2: 1, 3: 1, 4: 1, 5: 1, 6: 1, 7: 1, 8: 1, 9: 1, 10: 1, 11: 1, 12: 1, 13: 1, 14: 1, 15: 1, 16: 1, 17: 1, 18: 1, 19: 1, 20: 1, 21: 1, 22: 1, 23: 1, 24: 1, 25: 1, 26: 1, 27: 1, 28: 1, 29: 1, 30: 2, 31: 1, 32: 1, 33: 1, 34: 1, 35: 1, 3

# Sec 2: SVM

## Subsec 2.1.

Train a SVM using `sklearn`. Note that the hyperparameter tuning should be done using the "validation" set. **(P2-1-1: 20 points)**

In [198]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV


svm_model = SVC()

param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf'],
    'gamma': ['scale', 'auto']
}

grid_search = GridSearchCV(svm_model, param_grid, cv=5)
grid_search.fit(X_train, y_train)


best_svm_model = grid_search.best_estimator_
best_svm_model.fit(X_train, y_train)

accuracy = best_svm_model.score(X_test, y_test)
print("Accuracy on test set:", accuracy)

Accuracy on test set: 1.0


Report the accuracy and F1-score of the classifier on the testing set **(P2-1-2: 10 points)**

In [199]:
from sklearn.metrics import accuracy_score, f1_score

y_pred = best_svm_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy on test set:", accuracy)

f1 = f1_score(y_test, y_pred, average='weighted')
print("F1-score on test set:", f1)


Accuracy on test set: 1.0
F1-score on test set: 1.0


## Subsec 2.2.

Train a SVM from scratch using the following class **(P2-2-1: 40 points)**

Note: For this part, you are not required to use the validation set for hyperparameter tuning. However, you might get better results if you do that.

In [200]:
!pip install cvxopt

In [201]:
from cvxopt import matrix, solvers

class SVM:
    def __init__(self):
        self.W = None
        self.b = None

    def fit(self, X, y):
        y = np.where(y == 0, -1, 1)
        n_samples, n_features = X.shape

        P = matrix(np.outer(y, y) * np.dot(X, X.T))
        q = matrix(-np.ones(n_samples))
        G = matrix(-np.eye(n_samples))
        h = matrix(np.zeros(n_samples))
        A = matrix(y, (1, n_samples), tc='d')
        b = matrix(0.0)

        solvers.options['show_progress'] = False
        solution = solvers.qp(P, q, G, h, A, b)

        alpha = np.array(solution['x']).reshape(n_samples)

        sv_indices = alpha > 1e-5
        self.support_vectors = X[sv_indices]
        self.support_labels = y[sv_indices]
        self.support_alpha = alpha[sv_indices]

        self.W = np.dot(self.support_alpha * self.support_labels, self.support_vectors)
        self.b = np.mean(self.support_labels - np.dot(self.W, self.support_vectors.T))

        return self

    def transform(self, X):
        if self.W is None or self.b is None:
            raise ValueError("SVM not trained yet.")

        y_pred = np.sign(np.dot(X, self.W) + self.b)
        y_pred = np.where(y_pred == -1, 0, 1)

        return y_pred


In [202]:
svm = SVM()
svm.fit(X_train, y_train)
y_pred = svm.transform(X_test)

Report the accuracy and F1-score of the classifier on the testing set **(P2-2-2: 10 points)**

In [203]:
from sklearn.metrics import accuracy_score, f1_score

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

f1 = f1_score(y_test, y_pred, average='weighted')
print("F1-score:", f1)

Accuracy: 0.05
F1-score: 0.026250000000000002


نمی‌دونم چشه که به مشکل میخوره هی ولی فکر کنم دارم ایکس‌های اشتباهی رو به برچسب ها میدم ولی حل نشد آخرش مشکلم هرچی سعی کردم.

# Sec 3: PCA

Fetch all images taken from `jones`, `taylor`, `anderson` and `wilson` (use the `labelencoder` object). Perform PCA from scratch using the following `class` to reduce the dimension of those images to 3. Save the results as a `numpy` array of shape `(40, 3)`. **(P3-1: 30 points)**

In [204]:
encoded_labels = label_encoder.transform(metadata['label'])

target_labels = ['jones', 'taylor', 'anderson', 'wilson']

target_images = x_normalized[np.isin(encoded_labels, label_encoder.transform(target_labels))]

print("Shape of target images:", target_images.shape)


Shape of target images: (40, 4096)


In [205]:
class PCA:
    def __init__(self, n_components):
        self.n_components = n_components
        self.components_ = None

    def fit(self, X):
        self.components_ = None
        mean = np.mean(X, axis=0)
        centered_X = X - mean
        cov_matrix = np.cov(centered_X.T)
        eigenvalues, eigenvectors = np.linalg.eig(cov_matrix)

        sorted_indices = np.argsort(eigenvalues)[::-1]
        sorted_eigenvalues = eigenvalues[sorted_indices]
        sorted_eigenvectors = eigenvectors[:, sorted_indices]

        self.components_ = sorted_eigenvectors[:, :self.n_components]

        return self

    def transform(self, X):
        assert self.components_ is not None

        transformed_X = np.dot(X, self.components_)

        return transformed_X


In [206]:
pca = PCA(n_components=3)

pca.fit(target_images)
reduced_images = pca.transform(target_images)

print("Shape of reduced images:", reduced_images.shape)


Shape of reduced images: (40, 3)


Use `plotly` to plot these 3D samples. You should use their labels as the determiner of their color. **(P3-2: 10 points)**

In [207]:
!pip install plotly

In [208]:
import plotly.express as px

reduced_images_real = np.real(reduced_images)
reduced_images_imag = np.imag(reduced_images)

df = pd.DataFrame({
    'Component 1': reduced_images_real[:, 0],
    'Component 2': reduced_images_real[:, 1],
    'Component 3': reduced_images_real[:, 2],
    'Label': label_encoder.inverse_transform(encoded_labels[np.isin(encoded_labels, label_encoder.transform(target_labels))])
})

fig = px.scatter_3d(df, x='Component 1', y='Component 2', z='Component 3', color='Label', title='PCA Visualization')

fig.update_layout(
    scene=dict(
        xaxis=dict(title='Component 1'),
        yaxis=dict(title='Component 2'),
        zaxis=dict(title='Component 3')
    )
)

fig.show()


# Sec 4: K-Means

Using K-means, we want to perform clustering on the 3D samples. Consider 4 clusters. After running the K-means algorithm on the data, determine to which cluster each sample belongs. Then, use `plotly` to plot the 3D samples colored with respect to their clusters. **(P4-1: 15 points)**

In [209]:
import plotly.express as px

class KMeansClustering:
    def __init__(self, n_clusters):
        self.n_clusters = n_clusters
        self.kmeans = KMeans(n_clusters=n_clusters)

    def fit(self, x):
        self.kmeans.fit(x)

    def predict(self, x):
        return self.kmeans.predict(x)

    def plot_clusters(self, x):
        labels = self.predict(x)
        df = pd.DataFrame({'x': x[:, 0], 'y': x[:, 1], 'z': x[:, 2], 'label': labels})
        fig = px.scatter_3d(df, x='x', y='y', z='z', color='label')
        fig.show()

kmeans_clustering = KMeansClustering(n_clusters=4)
kmeans_clustering.fit(x)
kmeans_clustering.plot_clusters(x)


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



Measure the quality of the clustering using "silhoutte score" **(P4-2: 5 points)**

Note: You can use `sklearn` or other packages to calculate the score.

In [210]:
from sklearn.metrics import silhouette_score

labels = kmeans_clustering.predict(x)

silhouette_avg = silhouette_score(x, labels)
print("Silhouette Score:", silhouette_avg)


Silhouette Score: 0.10638006689429788


Compare the result of the clustering with the original labels. To be more precise, in this part, you should clearly answer the following questions with details:<br>

- Does each cluster represent a true label?
- What is your conclusion?

**(P4-3: 10 points)**

# Sec 5: Neural Net (Optional with Bonus Points)

Reshape the images so that they become of shape `64x64` **(P5-1: 5 points)**

In [211]:
resized_images = []
for file in metadata["path"]:
    image_path = f"images/content/img-data/{file}"
    image = Image.open(image_path)
    resized_image = image.resize((64, 64))
    resized_image = np.array(resized_image)
    resized_images.append(resized_image)

resized_images = np.array(resized_images)

In [212]:
resized_images.shape

(400, 64, 64)

Use different types of augmentation techniques (such as Random Flipping, Random Rotation, etc.) to increase the size of the training dataset. **(P5-2: 10 points)**

In [213]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    vertical_flip=True
)

x_train_augmented = X_train.reshape(X_train.shape[0], 64, 64, -1)
datagen.fit(x_train_augmented)

augmented_images = datagen.flow(x_train_augmented, y_train, batch_size=X_train.shape[0], shuffle=True)

x_train_augmented = augmented_images[0][0]
y_train_augmented = augmented_images[0][1]

x_train_augmented = x_train_augmented.reshape(x_train_augmented.shape[0], -1)

In [214]:
x_train_augmented

array([[0.39614412, 0.39616206, 0.39617997, ..., 0.7915378 , 0.7689369 ,
        0.7797736 ],
       [0.3673009 , 0.37227273, 0.3772446 , ..., 0.17625885, 0.15723988,
        0.20392157],
       [0.2285567 , 0.22731319, 0.2260697 , ..., 0.19094667, 0.18845968,
        0.18710399],
       ...,
       [0.5722863 , 0.51850355, 0.46333903, ..., 0.8381066 , 0.8689399 ,
        0.76807314],
       [0.5295317 , 0.5555153 , 0.5241736 , ..., 0.14108036, 0.13998573,
        0.13889109],
       [0.76488316, 0.777109  , 0.7909764 , ..., 0.38843408, 0.37098774,
        0.362267  ]], dtype=float32)

Using the augmented training set, train a neural network consisting of 2D convolutional layers. The exact architecture of the network is arbitrary. However, the last layer should have the "softmax" function as its activation. Also, using "max pooling" layers after convolutional ones is advised.

Note: Don't get frustrated if your network does not appear to be a good one. Give it some time to be trained. For instance, you might need to train it for 100 epochs. Make sure to try that out before quitting!

**(P5-3: 20 points)**

In [215]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 1)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(3, activation='softmax'))

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

x_train_augmented = x_train_augmented.reshape(x_train_augmented.shape[0], 64, 64, 1)

model.fit(x_train_augmented, y_train_augmented, epochs=100, batch_size=32, validation_data=(X_val, y_val))

x_test_normalized = X_test.reshape(X_test.shape[0], 64, 64, 1)
test_loss, test_accuracy = model.evaluate(x_test_normalized, y_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)


Epoch 1/100


InvalidArgumentError: ignored

Measure the accuracy of the network on the validation and testing datasets **(P5-4: 5 points)**